# Sentinel-2 based object detection (ships and parked planes)

This Jupyter notebook aims at showing the steps implemented to run teh Ships and Airplanes detection algorithm.
The detection algorithms are base on AI neural network and have been trained with more than 200 dedicated training datasets. Then they have been implemented as Headless notebook, and thus it has the following characteristics:

Stateless execution (no need of having a stable connection all the time while the script is running)
Easy to share the algorithm progress without having to share the source code 
Run via postman specifying script path and parameters
Easy to run complex scripts that take long time to run such as AI training algorithms

In this context the Ship detection and Airplane detection agorithms are under service licence, thus the complete workflow is not executable. For this reason we are showing here the necessary steps and parameters to interact with the Headless notebook, without the possibility to run the detections.

The approach implemented for this type of computation is based on the following 2 main step of workflow:

STEP 1: Set-up and input acquisition
- Import of necessary libraries
- set up of generic input parameters
    - Area of Interest - AOI (wkt polygon)
    - indicator code
    - sensor
    - overalpping AOI %
    - Cloud coverage % (global, over the entire image)
    - time range of interest
- set up of credentials for accessing and using EDC services
- core for generating the parameters necessary for triggering the AI algorithm:
    - generic input parameters
    - filtering of S2 images
        - covering the specific time range
        - over defined AOI with overlapping %
        - with defined Cloud Coverage % (global, over the entire image)
        - sensor

STEP 2: Pre-processing and object detection task execution
- Filtering S2 images according to cloudiness over AOI
- Send the encoded package to WPS endpoint for triggering the object detection algorithm (DL-based)
    - creation of CSV ready for timeseries ingestion into GeoDB
    - creation of geojson file with detection polygons
- Removing cloud-covered detected objects
      

The Following JN will cover and demonstrate the STEP 1 for the preparation fo the necessary parameters and the filtering of the Sentinel 2 Images that match the requirements specified in the generic input parameters

# STEP 1: Set-up and input acquisition

# Import of necessary libraries

In [1]:
# Import packages
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
import requests
import csv
import base64
import datetime
from shapely import wkt, geometry
#from shapely.geometry import shape, polygon
from shapely.geometry.multipolygon import MultiPolygon
from collections.abc import Iterable
from xml.etree import ElementTree
from IPython.display import clear_output
import geojson

# set up of credentials for accessing and using EDC services
- <ins>aoisfile</ins>: is a csv file with information about extended name of the Location (text), the AOI_ID (is a text with the identified code specific for that location), the economic indicator code (is a text with the identified code for the specific indicator), AOI Polygon (is a simplified polygon for intersecting the Sentinel 2 images covering the AOI, since there is a limitation on the points to be used for this computation, it is reccomended to use a rectangular or very simplified polygon), AOI subPolygon is the exact polygon of the AOI in which the algorithm will detect the features, it is reccomended to provide a polygon as much detailed as possible in order to reduce false positive outside the AOI).
- <ins>indicator</ins>: The indicator code that shall match one of the indicator code avaialbe in the aoisfile
- <ins>sensor</ins>: currently it has been implemented for Sentinel 2
- <ins>SENTINEL_2_L2A_collection_id</ins>: the collection ID as in sentinel HUB (default value = DSS2)
- <ins>overlap_perc</ins>: percetange of overalpping between Senteinel 2 image and the AOI polygon
- <ins>max_cc</ins>: the max cloud coverage over which the Sentinel 2 scene will be discarded 
- <ins>fromdate</ins>: start of the Time range for which we want to select Sentienl 2 scenese
- <ins>todate</ins>: end of the Time range for which we want to select Sentienl 2 scenese

In [2]:
# Parameters
aoisfile = "./Inputs/ships/Ships_Input_for_WPS_call.csv"
indicator = 'E200'
sensor = 'sentinel2'
SENTINEL_2_L2A_collection_id = "DSS2"
overlap_perc = 80
max_cc = 20
fromdate = "2022-01-01T00:00:00"
todate = "2022-04-20T23:59:05"

# Set UP of the EDC credentials

In [ ]:
import os

client_id = os.environ['SH_CLIENT_ID']
client_secret = os.environ['SH_CLIENT_SECRET']

In [3]:
token_url ='https://services.sentinel-hub.com/oauth/token'
token_info = "https://services.sentinel-hub.com/oauth/tokeninfo"

def get_token (client_id, client_secret, token_url, token_info):
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)
    token = oauth.fetch_token(token_url=token_url,
                              client_id=client_id, client_secret=client_secret)
    resp = oauth.get(token_info)
    return oauth

oauth = get_token (client_id, client_secret, token_url, token_info)

In [ ]:
# Input file
# aoisfile = "./Inputs/ships/Ships_Input_for_WPS_call.csv"
# TODO: creating a pull request on https://github.com/eurodatacube/notebooks/tree/master/notebooks/contributions/example_data
aoisfile = os.environ['EDC_PATH'] +  "notebooks/contributions/example_data/Ships_Input_for_WPS_call.csv"

# Core for generating the parameters necessary for triggering the AI algorithm

The following code will filter Sentinel 2 scenes according to paramters, displaying information of the AOI_ID, the overlapping percentage, the Cloud Coverage percentage and then providing the dates corresponding to avaialbe Sentienl 2 scenes matching the set parameters.

The output list could contains duplicated dates for which 2 sentinel images with two different UTM zones. In this case it is filtered on the first image with lowest cloud coverage. This approach has been implemented since the AI algortihm run on each provided image, that if not filtered, will cause duplicated counts on the same AOI and date but with different time.  

In [4]:
basesearch_url = f"https://creodias.sentinel-hub.com/ogc/wfs/e85faeeb-72ce-4de6-984c-c6dd0ede3ee0?"
with open(aoisfile, "r") as f:
    csv_reader = csv.reader(f, delimiter=",")
    for i,line in enumerate(csv_reader):
        count = 0
        if i == 0:
            continue
        dates=[]
        indicator_from_file = line[2]
        if indicator == indicator_from_file:
            aoiid = line[1]
            #print(line[1])

            pol = wkt.loads(str(line[3]))
            pol_subAOI = wkt.loads(str(line[4]))
            #Multipolyg to Polyg
            if isinstance(pol, Iterable):
                polyg = list(pol)[0].wkt
            else:
                polyg = pol.wkt
            g1 = wkt.loads(polyg)
            geom1 = geometry.shape(g1)
            offset=0
   
            multi_poly = str(MultiPolygon([geom1]))
            print('AoI id:', aoiid)
            if aoiid == 'EG1':
                overlap_perc = 30
            while True:
                search_url = f"{basesearch_url}\
service=WFS&version=1.0.0&request=GetFeature&\
typenames={SENTINEL_2_L2A_collection_id}&\
outputformat=application/json&\
FEATURE_OFFSET={offset}&\
GEOMETRY={polyg}&\
time={fromdate}/{todate}&srsname=EPSG:4326&\
maxcc={max_cc}"

                try:
                    response = oauth.get(search_url)
                    response.raise_for_status()
                    results = response.json()
                except requests.exceptions.HTTPError as e:
                    print (f"Error {e.response.content}")
                    break      
                feats = len(results['features'])
                if feats == 0:
                    break
                    #pagination
                count = count+feats
                offset = offset+100
                for subfeature in results['features']:
                    geom2 = geometry.shape(subfeature['geometry'])
                    overlap = geom1.intersection(geom2).area/geom1.area*100
                    ftime_out = subfeature['properties']['time']
                    print('overlap: ', overlap)
                    if overlap >= overlap_perc:
                        fdate = subfeature['properties']['date']
                        if fdate in dates and aoiid != "EG1" and aoiid != "EG2":
                            print ("Found duplicate")
                            continue
                        dates.append(fdate)
                        ftime = subfeature['properties']['time']
                        

print('\nFiltered dates:\n', dates)

AoI id: DE1
overlap:  85.91614427870401
overlap:  100.0
overlap:  78.75822669538235
overlap:  100.0
overlap:  80.99404086486553
overlap:  80.99405280412661
overlap:  100.0
overlap:  80.8577762986828
overlap:  100.0
overlap:  100.0
overlap:  84.50087594168775
overlap:  100.0
AoI id: IT3
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
AoI id: PL1
overlap:  97.55516173515117
overlap:  95.75311382204279
overlap:  96.2841148465969
overlap:  98.59752459330083
overlap:  100.0
overlap:  98.7028074774188
overlap:  96.93688259247746
overlap:  100.0
overlap:  100.0
AoI id: FR3
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
overlap:  100.0
AoI id: BE3
overlap:  100.0
overlap:  0.0
overlap:  100.0
overlap:  0.0
overlap:  100.0
overlap:  0.0
overlap:  100.0
overl

# STEP 2: Pre-processing and object detection task examples

As previously mentioned, the cloud and object detection (ships and airplanes) algorithm is under service licence and the complete workflow is thus not executable.
For this reason we are showing here some representative examples of the outputs generated by these algorithms. 

# Cloud detection module
Features
- Performing cloud detection over AOI
- Assessing percentage of clouds (PoC) and filtering out cloudy images (above a specified cloudiness threshold)
- Removing cloud-covered detected objects (post-processing step)

In [10]:
# import image module
from IPython.display import Image
  
# get the image
Image(url="images/Ghent_cloud_detection.jpg", width=600, height=600)

In [11]:
# get the image
Image(url="images/Suez_cloud_detection.jpg", width=600, height=600)

# Object detection module

In [12]:
# get the image
Image(url="images/Ghent.jpg", width=600, height=600)

In [14]:
Image(url="images/Dunkirque_1.jpg", width=600, height=600)